# MC866 Project 1

## PART I - Clustering Methods

### K-Means
API based on sklearn implementation of K-Means

In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
class KMeans:

    def __init__(self, n_clusters=8,  max_iter=300, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.rand = np.random.RandomState(random_state)

    def fit(self, X):
        # Selecting random centroids
        centroids = self.rand.permutation(X)[:self.n_clusters] # the centroids are selected from random points
        



In [3]:
X = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

k = KMeans(n_clusters=2)
k.fit(X)

In [6]:
df = pd.read_csv("./data/cluster.dat", sep=' ', header=None)
df.head()

,0,1
0,1555.0,28.65
1,1490.0,27.55
2,1445.0,28.35
3,1415.0,28.80
4,1375.0,28.05
